In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from google.colab import drive
drive.mount("/content/MyDrive/")

In [ ]:
import sys
import random
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, TimeDistributed, Dense
from tensorflow.keras.optimizers import Adam

gdrive_path = "/content/MyDrive/MyDrive/Research/IQ-Learn_CF/Prog/"
sys.path.append(gdrive_path)
from CF_Env import CF_Env

# Set hyper-parameters

In [ ]:
EPOCHS = 500
BATCH_SIZE = 64
INPUT_SIZE = 3
OUTPUT_SIZE = 1
CELL_SIZE = 64
LEARNING_RATE = 5e-5
env = CF_Env()

# Load training and testing dataset

In [ ]:
train_set = np.load(gdrive_path+'CF_HV-AV_4/train_cf_4.npy', allow_pickle=True)
training = train_set.item()['states']
training = np.array([np.array(x) for x in training])
print('Number of training samples:', len(training))

test_set = np.load(gdrive_path+'CF_HV-AV_4/test_cf_4.npy', allow_pickle=True)
testing = test_set.item()['states']
testing = np.array([np.array(x) for x in testing])
print('Number of testing samples:', len(testing))

# Customize the loss function

In [ ]:
def space_error(y_true, y_pred): # y_true: RealSpaceData; y_pred: action
    global state, next_state, done, action, timestep
    
    # update sv speed
    sv_next_spd = state[1] + 0.1*y_pred
    # calculate sv distance
    sv_dist = (state[1] + sv_next_spd)*0.1/2
    # update lv speed
    lv_next_spd = env.LVSpdData[timestep]
    # calculate lv distance
    lv_dist = (env.LVSpdData[timestep-1] + lv_next_spd)*0.1/2
    # update spacing
    sim_space = state[0] - sv_dist + lv_dist
    # update reletive speed
    rel_spd = sv_next_spd - lv_next_spd

    # update state
    next_state = [sim_space, sv_next_spd, rel_spd]

    # check crash
    done = True if sim_space < 0 else False

    # calculate error    
    err = tf.square(y_true - sim_space) / tf.square(y_true)
    return err

# Define the LSTM model

In [ ]:
# build LSTM model
model = Sequential()
model.add(LSTM(
    batch_input_shape=(BATCH_SIZE, None, INPUT_SIZE),
    units=CELL_SIZE, # dimensionality of the output space.
    return_sequences=True,  # True: output at all steps. False: output as last step.
    dropout=0, # Fraction of the units to drop for the linear transformation of the inputs.
    recurrent_dropout=0 # Fraction of the units to drop for the linear transformation of the recurrent state.
))
model.add(LSTM(
    batch_input_shape=(BATCH_SIZE, None, INPUT_SIZE),
    units=CELL_SIZE, # dimensionality of the output space.
    return_sequences=True,  # True: output at all steps. False: output as last step.
    dropout=0, # Fraction of the units to drop for the linear transformation of the inputs.
    recurrent_dropout=0 # Fraction of the units to drop for the linear transformation of the recurrent state.
))
model.add(TimeDistributed(Dense(OUTPUT_SIZE)))
# compile model
model.compile(optimizer=Adam(LEARNING_RATE), loss=space_error)
model.summary()

# Calculate Root Mean Square Percentage Error

In [ ]:
def cal_rmspe(y_true, y_pred):
    # Compute Root Mean Square Percentage Error between two arrays
    loss = np.sqrt(np.sum(np.square(y_true - y_pred))/np.sum(np.square(y_true)))

    return loss

# Begin training

In [ ]:
# # store the training loss
# lstm_train = []

# # save training records
# f = open(gdrive_path+"CF_HV-AV_4/LSTM/train_LSTM_log.txt", "w")

# # define early stop criterion
# epoch = 0
# max_iter = 50
# tolerance_iter = 0
# min_loss = []

# for epoch in range(EPOCHS):
#   init_data = np.array(train_set.item()['states'][epoch%len(training)])
#   state = env.reset(init_data)

#   """start training"""
#   for timestep in range(1, len(init_data)):
#     action = model.predict(state.reshape(1, -1, INPUT_SIZE), BATCH_SIZE)
#     loss = model.train_on_batch(state.reshape(1, -1, INPUT_SIZE), env.RealSpaceData[timestep].reshape(1, 1, 1))
#     lstm_train.append(loss)
#     next_state = [i.numpy() for i in next_state]

#     # next_states
#     a = []
#     for item in next_state:
#       if isinstance(item, np.ndarray):
#         a.append(item[0][0][0])
#       else:
#         a.append(item)
#     next_state = np.array(a)
#     state = next_state

#     if done:
#         break

#   """start testing"""
#   # store simulated and real data
#   # spacing
#   SimSpaceData = []
#   RealSpaceData = []
#   # following speed
#   SimSpeedData = []
#   RealSpeedData = []

#   for evl_epoch in range(len(testing)):
#     init_data = np.array(train_set.item()['states'][evl_epoch])
#     state = env.reset(init_data)

#     # store simulated spacing and following speed
#     SimSpaceData.append(state[0])
#     SimSpeedData.append(state[1])

#     # store real spacing and following speed
#     RealSpaceData.append(env.RealSpaceData[0])
#     RealSpeedData.append(env.RealSpeedData[0])

#     while True:
#       action = model.predict(state.reshape(1, -1, INPUT_SIZE), BATCH_SIZE)
#       next_state, reward, done, _ = env.step(action)

#       # next_states
#       a = []
#       for item in next_state:
#         if isinstance(item, np.ndarray):
#           a.append(item[0][0][0])
#         else:
#           a.append(item)
#       next_state = np.array(a)
#       state = next_state

#       if done:
#           break

#       # store simulated spacing and following speed
#       SimSpaceData.append(state[0])
#       SimSpeedData.append(state[1])
#       # store real spacing and following speed
#       RealSpaceData.append(env.RealSpaceData[env.timeStep-1])
#       RealSpeedData.append(env.RealSpeedData[env.timeStep-1])

#   # spacing
#   SimSpaceData = np.array(SimSpaceData)
#   RealSpaceData = np.array(RealSpaceData)
#   # following speed
#   SimSpeedData = np.array(SimSpeedData)
#   RealSpeedData = np.array(RealSpeedData)

#   print('Vali\tEpoch {}\tSpacing RMSPE: {:.6f}\tSpeed RMSPE: {:.6f}'.format(
#       epoch, cal_rmspe(y_true=RealSpaceData, y_pred=SimSpaceData), cal_rmspe(y_true=RealSpeedData, y_pred=SimSpeedData)))
#   f.write('Vali\tEpoch {}\tSpacing RMSPE: {:.6f}\tSpeed RMSPE: {:.6f}\n'.format(
#       epoch, cal_rmspe(y_true=RealSpaceData, y_pred=SimSpaceData), cal_rmspe(y_true=RealSpeedData, y_pred=SimSpeedData)))
#   f.write('\n')
# # 
#   # check if need to stop training
#   if len(min_loss) == 0:
#     min_loss.append(cal_rmspe(y_true=RealSpaceData, y_pred=SimSpaceData))
#   else:
#     # check if rmspe of spacing is decreasing
#     if min_loss[-1] < cal_rmspe(y_true=RealSpaceData, y_pred=SimSpaceData):
#       tolerance_iter += 1
#     else:
#       # keep the lower rmspe of spacing
#       min_loss[-1] = cal_rmspe(y_true=RealSpaceData, y_pred=SimSpaceData)
#       # reinitialize the pointer
#       tolerance_iter = 0
#       # save the best model
#       final_model = model
#       final_model.save(gdrive_path+'CF_HV-AV_4/LSTM/LSTM.h5') # save the best model

#   if tolerance_iter >= max_iter:
#     break

# np.save(gdrive_path+'CF_HV-AV_4/LSTM/train_LSTM_loss.npy', lstm_train)
# f.close()

# Save model

In [ ]:
# model.save(gdrive_path+'CF_HV-AV_4/LSTM/LSTM.h5') # save the final trained model

# Load the saved model

In [ ]:
# Don't compile while loading as custom objects are not serialized
LSTM = tf.keras.models.load_model(gdrive_path+'CF_HV-AV_4/LSTM/LSTM.h5', custom_objects={space_error:space_error}, compile=False)

# compile the loaded model with the custome objects
LSTM.compile(optimizer=Adam(LEARNING_RATE), loss=space_error)

# Begin testing

In [ ]:
# store the simulated trajectories
lstm_test = []

# store simulated and real data
# spacing
SimSpaceData = []
RealSpaceData = []
# following speed
SimSpeedData = []
RealSpeedData = []

for epoch in range(len(testing)):
  init_data = np.array(test_set.item()['states'][epoch])
  state = env.reset(init_data)
  traj = [state]

  # store simulated spacing and following speed
  SimSpaceData.append(state[0])
  SimSpeedData.append(state[1])
  # store real spacing and following speed
  RealSpaceData.append(env.RealSpaceData[0])
  RealSpeedData.append(env.RealSpeedData[0])

  while True:
    action = LSTM.predict(state.reshape(1, -1, INPUT_SIZE), BATCH_SIZE)
    next_state, reward, done, _ = env.step(action)

    # next_states
    a = []
    for item in next_state:
      if isinstance(item, np.ndarray):
        a.append(item[0][0][0])
      else:
        a.append(item)
    next_state = np.array(a)
    state = next_state
    traj.append(state)

    if done:
        break

    # store simulated spacing and following speed
    SimSpaceData.append(state[0])
    SimSpeedData.append(state[1])

    # store real spacing and following speed
    RealSpaceData.append(env.RealSpaceData[env.timeStep-1])
    RealSpeedData.append(env.RealSpeedData[env.timeStep-1])
   
  traj = np.array(traj)
  lstm_test.append(traj)

# spacing
SimSpaceData = np.array(SimSpaceData)
RealSpaceData = np.array(RealSpaceData)
spacing_rmspe = cal_rmspe(y_true=RealSpaceData, y_pred=SimSpaceData)
# following speed
SimSpeedData = np.array(SimSpeedData)
RealSpeedData = np.array(RealSpeedData)
speed_rmspe = cal_rmspe(y_true=RealSpeedData, y_pred=SimSpeedData)

print('TEST\tTotal\tAverage Spacing RMSPE: {:.6f}\tAverage Speed RMSPE: {:.6f}'.format(spacing_rmspe, speed_rmspe))

np.save(gdrive_path+'CF_HV-AV_4/LSTM/test_LSTM_traj.npy', lstm_test)
f.close()

# Visualize the simulated results

In [ ]:
train_loss = np.load(gdrive_path+'CF_HV-AV_4/LSTM/train_LSTM_loss.npy', allow_pickle=True)
test_traj = np.load(gdrive_path+'CF_HV-AV_4/LSTM/test_LSTM_traj.npy', allow_pickle=True) # spacing, relative speed and following vehicle speed

In [ ]:
plt.plot(train_loss)
plt.ylim(0, 1)

In [ ]:
# spacing
plt.plot(test_traj[0][:, 0], label ='Sim Spacing')
plt.plot(testing[0][:, 0], label ='Real Spacing')
plt.legend()

In [ ]:
# following speed
plt.plot(test_traj[0][:, 1], label ='Sim FV Speed')
plt.plot(testing[0][:, 1], label ='Real FV Speed')
plt.legend()